In [15]:
import pandas as pd
import json

Inception v3 trained on balanced dataset, focal loss $\gamma = 2$, 5 epochs, 100 batch size, Adam lr=0.0005

In [16]:
# path = "../results/inception_v3/fl/"
path = "../results/AlexNet/xen/"

In [17]:
df = pd.read_csv(path+"avg_metrics.csv")

In [18]:
df.columns

Index(['image_idxs', 'accuracy', 'precision', 'specificity', 'sensibility',
       'IoU', 'dice_coeff', 'f1_score'],
      dtype='object')

In [19]:
# Function to parse indices string into list of ints
def parse_json_column(json_str):
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return None

# Apply the parsing function to the 'data' column
df["image_idxs"] = df["image_idxs"].apply(parse_json_column)

In [20]:
df

,image_idxs,accuracy,precision,specificity,sensibility,IoU,dice_coeff,f1_score
0,"[0, 1, 2]",0.919238,0.919170,0.702801,0.980183,0.902480,0.948644,0.794920
1,"[3, 4, 5]",0.880106,0.870127,0.549327,0.983402,0.857717,0.923161,0.665914
2,"[6, 7, 8]",0.958229,0.958932,0.366178,0.997110,0.956290,0.977608,0.484766
3,"[9, 10, 11]",0.917008,0.913916,0.494735,0.995401,0.910052,0.952885,0.639135
4,"[12, 13, 14]",0.944509,0.945661,0.421313,0.995233,0.941425,0.969734,0.567903
5,"[15, 16, 17]",0.880958,0.877920,0.617178,0.954530,0.844732,0.914410,0.711863
6,"[18, 19, 20]",0.952462,0.951943,0.609916,0.996094,0.948419,0.973468,0.740835
7,"[21, 22, 23]",0.883123,0.878553,0.428735,0.953104,0.845483,0.913152,0.478037
8,"[24, 25, 26]",0.930773,0.921518,0.145587,0.996479,0.918565,0.953929,0.185312
9,"[27, 28]",0.917656,0.936738,0.634230,0.963887,0.904863,0.949920,0.744507


In [21]:
metrics_df = df.drop(columns=['image_idxs'])

In [22]:
cols = metrics_df.columns
n_samples, n_features = metrics_df.shape

In [23]:
list(cols)

['accuracy',
 'precision',
 'specificity',
 'sensibility',
 'IoU',
 'dice_coeff',
 'f1_score']

In [24]:
means = metrics_df.mean()
means

accuracy       0.918406
precision      0.917448
specificity    0.497000
sensibility    0.981542
IoU            0.903003
dice_coeff     0.947691
f1_score       0.601319
dtype: float64

In [25]:
stds = metrics_df.std()/n_samples
stds

accuracy       0.002924
precision      0.003236
specificity    0.016402
sensibility    0.001799
IoU            0.004140
dice_coeff     0.002362
f1_score       0.018204
dtype: float64

In [26]:
n_decimals = 4
cross_val_data = [f"{means.iloc[i]:0.{n_decimals}f} +- {stds.iloc[i]:0.{n_decimals}f}" for i in range(n_features)]

In [27]:
cross_val_metrics_df = pd.DataFrame([cross_val_data], columns=tuple(cols))
cross_val_metrics_df.loc[0]

accuracy       0.9184 +- 0.0029
precision      0.9174 +- 0.0032
specificity    0.4970 +- 0.0164
sensibility    0.9815 +- 0.0018
IoU            0.9030 +- 0.0041
dice_coeff     0.9477 +- 0.0024
f1_score       0.6013 +- 0.0182
Name: 0, dtype: object

In [28]:
cross_val_metrics_df.to_csv(path+"cross_val_metrics.csv", index=False, columns=tuple(cols))